In [1]:
%load_ext autoreload
%autoreload 2
import os,sys
import PATH
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import utils
from models import reader
from models.popen import Auto_popen

add /ssd/users/wergillius/Project/MTtrans to path
/ssd/users/wergillius/Project/MTtrans


In [22]:
torch.set_num_threads(2)

In [38]:
from evaluation.evaluation_func import *

In [46]:
from scipy import stats

## prepare data

In [4]:
datapopen = Auto_popen(os.path.join(utils.script_dir,  'log/Backbone/RL_hard_share/3R/schedule_MTL.ini'))

In [13]:
datapopen.shuffle = False
loaderset = kfold_load_data('log/Backbone/RL_hard_share/3R/schedule_MTL.ini', 'cpu', 1)

In [19]:
# try RP293T
RP293_train_df = loaderset['RP_293T'][0].dataset.df
RP293_train_df.head()

,T_id,te,utr,utr_len,num_index,te_scale,log_te,scaled_log_te,nupack_MFE,eGFP50_nupMFE,full_nupMFE
2570,ENST00000487505.1,0.588247,GGCTAAGTCAGCGCTTAATCAAGCGCCGGCTGGAGGCCCAGCCTGG...,100,9456,-0.067221,-0.530608,0.079781,-29.774862,-57.036186,-143.789932
5155,ENST00000393033.4,1.290938,AGGCCCGGTCTGGAGTTTGCGGAGGGCCGAGCCGGGTGCGCACGGG...,96,3312,0.087494,0.255369,0.821156,-43.807373,-57.952656,-167.165115
12078,ENST00000455162.1,0.099419,ATGAATGGGCGCTGGGAACACGCCCGCGAGGTGGGGACGCGCCGGC...,99,16359,-0.174848,-2.308414,-1.597138,-48.520916,-64.145287,-162.241669
3024,ENST00000436547.2,0.930060,AACCCTGCCCTCCCCCTTGTCCCGGGATCGCTCCGTCGCACCCAC,45,5941,0.008038,-0.072506,0.511886,-6.609412,-31.775995,-124.441010
4206,ENST00000542928.1,0.805883,CAATTTCTCCTCCCCCTCCCGGCCAA,26,7062,-0.019303,-0.215817,0.376709,0.000000,-24.539268,-118.347366


### 3m model kfold = 1

In [14]:
model_config = "log/Backbone/RL_hard_share/3M/small_repective_filed_strides1113.ini"

In [20]:
model = reload_model(model_config, 'cpu', 1)

In [26]:
model.all_tasks

['MPA_U', 'MPA_H', 'MPA_V']

In [57]:
def test_3m_on_3rdata(model, data_set):

    pred_3m = {}
    for task in model.all_tasks:
        model.task = task
        y_true, y_pred, _ = val_a_epoch(model, loaderset[data_set][0], 'cpu')
        pred_3m[task] = y_pred

    # postprocessing_params

    pred_3m = {k:standardize_by(v, k) for k, v in pred_3m.items()}
    pred_3m_df = pd.DataFrame(pred_3m)
    pred_3m_df[f'y_true_{data_set}'] = y_true

    pred_3m_df[f'y_true_{data_set}_scaled'] = standardize_by(y_true, data_set)

    pred_3m_df.mean(axis=0)

    for task in model.all_tasks:
        model.task = task
        cor= stats.spearmanr(pred_3m_df[task].values, pred_3m_df[f'y_true_{data_set}_scaled'].values)[0]
        rho= stats.pearsonr(pred_3m_df[task].values, pred_3m_df[f'y_true_{data_set}_scaled'].values)[0]
        print(task, "pearson r: %.3f"%rho, "spearman r: %.3f"%cor)
    return pred_3m_df

In [48]:
rp293t_result = test_3m_on_3rdata("RP_293T")

MPA_U pearson r: 0.091 spearman r: 0.135
MPA_H pearson r: 0.085 spearman r: 0.115
MPA_V pearson r: 0.150 spearman r: 0.154


In [55]:
rppc3_result = test_3m_on_3rdata("RP_PC3")

100%|██████████| 492/492 [00:24<00:00, 20.23it/s]

MPA_U pearson r: 0.157 spearman r: 0.204
MPA_H pearson r: 0.125 spearman r: 0.152
MPA_V pearson r: 0.213 spearman r: 0.206


In [56]:
muscle_result = test_3m_on_3rdata("RP_muscle")

100%|██████████| 47/47 [00:02<00:00, 18.78it/s]

MPA_U pearson r: 0.050 spearman r: 0.095
MPA_H pearson r: 0.069 spearman r: 0.112
MPA_V pearson r: 0.158 spearman r: 0.121
